# Let's take a look at the top ranked genes that Rachel's analysis discovered :)

In [2]:
top5_degree = [
    "BB_RS07040",
    "BB_RS05890",
    "BB_RS07880",
    "BB_RS05830",
    "BB_RS06360",
]
top5_betweenness = [
    "BB_RS05890",
    "BB_RS07040",
    "BB_RS06060",
    "BB_RS07105",
    "BB_RS06025",
]
top5_closeness = [
    "BB_RS07040",
    "BB_RS05890",
    "BB_RS07880",
    "BB_RS05830",
    "BB_RS06360",
]
top5_eigenvector = [
    "BB_RS07880",
    "BB_RS05830",
    "BB_RS07040",
    "BB_RS06350",
    "BB_RS07450",
]

top5 = set(top5_degree + top5_betweenness + top5_closeness + top5_eigenvector)
rank1 = [top5_degree[0], top5_betweenness[0], top5_closeness[0], top5_eigenvector[0]]
rank2 = [top5_degree[1], top5_betweenness[1], top5_closeness[1], top5_eigenvector[1]]
rank3 = [top5_degree[2], top5_betweenness[2], top5_closeness[2], top5_eigenvector[2]]
rank4 = [top5_degree[3], top5_betweenness[3], top5_closeness[3], top5_eigenvector[3]]
rank5 = [top5_degree[4], top5_betweenness[4], top5_closeness[4], top5_eigenvector[4]]
ranks = [rank1, rank2, rank3, rank4, rank5]

In [21]:
# now let's import and parse the GFF reference!
from Bio import SeqIO
import pandas as pd
import numpy as np
import pprint
import pickle
import matplotlib.pyplot as plt
from collections import defaultdict
from itertools import combinations


In [22]:
# specify paths
gff_file = "../../ref/GCF_000008685.2_ASM868v2_genomic.gff" # gff file
fasta_file = "../../ref/GCF_000008685.2_ASM868v2_genomic.fna" # fasta file
# and the acc2plasmid file
acc2plasmid = "../../plasmid_id/plasmid_seqs/acc_to_plasmid.tsv"

In [23]:
# read gff
with open(gff_file, "r") as handle:
    gff = handle.readlines()

# read acc2plasmid
acc2plasmid_df = pd.read_csv(acc2plasmid, sep="\t", header=0)
acc2plasmid_df.columns = ["acc", "plasmid", "strain"]
acc2name = dict(zip(acc2plasmid_df.acc, acc2plasmid_df.plasmid))
pprint.pprint(acc2name) # UGH ITS GENBANK ACCS!!! TIME TO PARSE THE FASTA I GUESS.

{'AE000783.1': 'chromosome',
 'AE000784.1': 'lp28-3',
 'AE000785.1': 'lp25',
 'AE000786.1': 'lp28-2',
 'AE000787.1': 'lp38',
 'AE000788.1': 'lp36',
 'AE000789.1': 'lp28-4',
 'AE000790.2': 'lp54',
 'AE000791.1': 'cp9',
 'AE000792.1': 'cp26',
 'AE000793.2': 'lp17',
 'AE000794.2': 'lp28-1',
 'AE001575.1': 'cp32-1',
 'AE001576.1': 'cp32-3',
 'AE001577.1': 'cp32-4',
 'AE001578.1': 'cp32-6',
 'AE001579.1': 'cp32-7',
 'AE001580.1': 'cp32-8',
 'AE001581.1': 'cp32-9',
 'AE001582.2': 'lp21',
 'AE001583.1': 'lp5',
 'AE001584.1': 'lp56',
 'CP001198.1': 'lp25',
 'CP001199.1': 'lp54',
 'CP001200.1': 'cp32-12',
 'CP001201.1': 'lp36',
 'CP001202.1': 'cp32-3+10',
 'CP001203.1': 'lp28-3',
 'CP001204.1': 'lp17',
 'CP001205.1': 'chromosome',
 'CP001206.1': 'cp32-1',
 'CP001207.1': 'lp28-4',
 'CP001208.1': 'cp32-4',
 'CP001209.1': 'lp28-2',
 'CP001210.1': 'lp28-1',
 'CP001211.1': 'cp32-9',
 'CP001212.1': 'cp26',
 'CP001256.1': 'lp28-3',
 'CP001257.1': 'lp54',
 'CP001258.1': 'lp17',
 'CP001259.1': 'cp32-6',

In [24]:

#pickle.dump(acc2name, open("acc2name_gb.pkl", "wb"))

In [25]:

# read fasta
with open(fasta_file, "r") as handle:
    fasta = list(SeqIO.parse(handle, "fasta"))

# parse fasta
acc2name_rs = {}
for record in fasta:
    acc = record.id.split(" ")[0]
    print(acc)
    name = record.description.split(",")[0].split(" ")[-1]
    print(name)
    acc2name_rs[acc] = name


NC_001318.1
B31
NC_000957.1
lp5
NC_001904.1
cp9
NC_001849.2
lp17
NC_000955.2
lp21
NC_001850.1
lp25
NC_001903.1
cp26
NC_001851.2
lp28-1
NC_001852.1
lp28-2
NC_001853.1
lp28-3
NC_001854.1
lp28-4
NC_000948.1
cp32-1
NC_000949.1
cp32-3
NC_000950.1
cp32-4
NC_000951.1
cp32-6
NC_000952.1
cp32-7
NC_000953.1
cp32-8
NC_000954.1
cp32-9
NC_001855.1
lp36
NC_001856.1
lp38
NC_001857.2
lp54
NC_000956.1
lp56


In [26]:
pprint.pprint(acc2name_rs)
#pickle.dump(acc2name_rs, open("acc2name_refseq.pkl", "wb"))


{'NC_000948.1': 'cp32-1',
 'NC_000949.1': 'cp32-3',
 'NC_000950.1': 'cp32-4',
 'NC_000951.1': 'cp32-6',
 'NC_000952.1': 'cp32-7',
 'NC_000953.1': 'cp32-8',
 'NC_000954.1': 'cp32-9',
 'NC_000955.2': 'lp21',
 'NC_000956.1': 'lp56',
 'NC_000957.1': 'lp5',
 'NC_001318.1': 'B31',
 'NC_001849.2': 'lp17',
 'NC_001850.1': 'lp25',
 'NC_001851.2': 'lp28-1',
 'NC_001852.1': 'lp28-2',
 'NC_001853.1': 'lp28-3',
 'NC_001854.1': 'lp28-4',
 'NC_001855.1': 'lp36',
 'NC_001856.1': 'lp38',
 'NC_001857.2': 'lp54',
 'NC_001903.1': 'cp26',
 'NC_001904.1': 'cp9'}


In [29]:
# parse the GFF file
genes = []
for line in gff:
    if line.startswith("#"):
        continue
    line = line.strip().split("\t")
    gene = line[-1].split(";")[1].split("=")[1].replace("gene-", "")
    start = int(line[3])
    end = int(line[4])
    genes.append((gene, start, end))

# sort the genes by start position
genes = sorted(genes, key=lambda x: x[1])

# create a dictionary of gene positions
gene_positions = {}
for gene, start, end in genes:
    gene_positions[gene] = (start, end)

# create a dictionary of gene lengths
gene_lengths = {}
for gene, start, end in genes:
    gene_lengths[gene] = end - start

# create a dictionary of gene names
gene_names = {}
for gene, start, end in genes:
    gene_names[gene] = gene


In [49]:
# create a dictionary of gene_names to descriptions
gene_products = {}
for line in gff:
    if line.startswith("#"):
        continue
    line = line.strip().split("\t")

    gene = line[-1].split(";")[1].split("=")[1].replace("gene-", "")
    desc = line[-1].split(";")[-4]#.replace("product=", "")
    name = line[-1].split(";")[1]#.replace("Name=", "")
    old_name = line[-1].split(";")[-1]  # this is the traditional nomenclature
    if old_name.startswith("old_locus_tag"):
        old_name = old_name.split("=")[1]
    else:
        old_name = "No Traditional Nomenclature!"
    replicon = line[0]
    # ok now make dictionary.
    gene_products[gene] = {
        'name': name,
        'trad_nomenclature': old_name,
        'replicon': (replicon, acc2name_rs[replicon]),
        'coords': gene_positions[gene],
        'product': desc,
    }


In [50]:
pprint.pprint(gene_products)
#pickle.dump(gene_products, open("b31_gene_dict.pkl", "wb"))

{'BB_RS00005': {'coords': (105, 677),
                'name': 'Parent=gene-BB_RS00005',
                'product': 'locus_tag=BB_RS00005',
                'replicon': ('NC_001318.1', 'B31'),
                'trad_nomenclature': 'No Traditional Nomenclature!'},
 'BB_RS00010': {'coords': (768, 1796),
                'name': 'Parent=gene-BB_RS00010',
                'product': 'locus_tag=BB_RS00010',
                'replicon': ('NC_001318.1', 'B31'),
                'trad_nomenclature': 'No Traditional Nomenclature!'},
 'BB_RS00015': {'coords': (1784, 3249),
                'name': 'Parent=gene-BB_RS00015',
                'product': 'locus_tag=BB_RS00015',
                'replicon': ('NC_001318.1', 'B31'),
                'trad_nomenclature': 'No Traditional Nomenclature!'},
 'BB_RS00020': {'coords': (3395, 5188),
                'name': 'Parent=gene-BB_RS00020',
                'product': 'locus_tag=BB_RS00020',
                'replicon': ('NC_001318.1', 'B31'),
                'trad

In [45]:
for gene in top5:
    print(gene, gene_products[gene])
print("")

for rank in ranks:
    for gene in rank:
        print(f"Rank {ranks.index(rank)+1}: {gene}, {gene_products[gene]}")
    print("")

KeyError: 'BB_RS07450'